# <span style="color: blue;">Analysing method related differences script - bioinformatic differences</span>

This script goes through the investigating discrepancies between the 4 methods

1. ABRicate (our baseline method)
2. ARIBA
3. KmerResistance
4. SRST2

**NOTE FOR EASE OF READING, SOME CODE IS REPEATED IN EACH SECTION, TO MAKE EACH SECTION BE A DISCREET UNIT OF ANALYSIS, THIS LENGTHANS THE RUNTIME OF THE CODE BUT MAKES IT EASIER TO UNDERSTAND. THE RUNTIME IS STILL SHORT (< 2 hours even comparing all database similarities)**

### <span style="color: green;">Highlighting database dependency.</span>

For each of several databases, it does the following:
1. reads in all the results we obtained
3. This is then used to produce **Figure X** 


Databases used
**Primary**
Resfinder 1st October 2019 release 
**Secondary**
* CARD 23rd October 2019 release
* beta-lactam.fsa of the Resfinder 1st October 2019 release
* aminoglycoside.fsa  of the Resfinder 1st October 2019 release
* quinolone.fsa of the Resfinder 1st October 2019 release
* trimethoprim.fsa of the Resfinder 1st October 2019 release
* sulphonamide.fsa of the Resfinder 1st October 2019 release
* Resfinder 22nd of January 2019 release
* Resfinder 22nd of January 2018 release
* Resfinder 26th of January 2017 release



From here on, we are investigating causes of error and only use the Resfinder 1st October 2019 release.

### <span style="color: green;">Identifying which TRGs are most commonly discrepant and then </span>

1. For this we first identify which reported TRGs likely represent the same gene found differently by the different programs (see image below)

![image](method_1.png)

2. We then look at patterns of discrepancy and identify the 10 most common across the data
3. We then demonstrate what we believe the cause of each of these is (simulated example in each cell of the notebook)





### <span style="color: green;">Highlighting extent of annotation errors</span>

This then goes through how we determine if an error is likely due to annotation or not

1. For each discrepant, we identify if any of the identified TRGs are complete within the assembly.
2. If so, we take the containing contig and simulate **perfect* reads at 50x and 500x from it (10 times each).
3. We then observe if the same misclassification pattern occurs again. If so the error is deemed likely annotation
4. This is then used to produce **Figure XX** in the paper.

Note for some of the code from this section is only presented here in markdown as I can't provide any full sequence files on GitHub. However, the sequences for this project are available on NCBI and the code can be recreated to analyse these scripts here.

### <span style="color: green;">Investigating non annotation errors.</span>

This section finally ends on investigation of 6 samples where different beta-lcatamase genes were reported but were not artefactual.





## <span style="color: blue;">Setup</span>

**So first steps are to load in required modules and then identify all the output reports**


#### Dependencies

1. Python 3 
2. Biopython
3. Pandas
4. Numpy
5. tqdm
6. networkx

#### Inputs
Some notes for this step, firstly which files we take
1. for ABRicate we take each contigs.tab file
2. for ARIBA we use its summary file
3. for KmerResistance we use the .KmerRes files
4. for SRST2 we use the .out__fullgenes__seqs_clustered__results.txt files

Note these were chosen as they seem to follow guidelines and where no guidelines available, give us the most closely matching results between the four programs

Note also SRST2 does not produce output if it finds no genes. The others all do

#### Resfinder database

For this we load
1. The naming link database
2. For each of the sub databases by antibiotic class 
  * beta-lactam
  * quinolone
  * aminoglycoside
  * sulfa antibiotics
  * trimethoprim
3. The whole database

We do 1 so the results are interpretable
We do 2 so we can breakdown results by antibiotic class as well as specific genes
We do 3 to do all versus all of the resistance database


In [1]:
# We start by importing required modules
# File structure
import os
import csv


# Pandas
import pandas as pd
import numpy as np
import networkx as nx
# These are the fundamental modules used for analysing the data


# Pictures
import matplotlib.pyplot as plt
import seaborn as sns

# Sequence manipulation
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# These modules are priarily used for loading the database, and then for identifying whether there are any
# Perfrect protein matches in the dataset


# Plotting


# Other general
from tqdm import tnrange, tqdm_notebook
# We just import this to check our code runs sensibly and to get timing estimates for stuff
from copy import deepcopy

#Looking at local files
%ls 



by_trg_pattern.csv            method_related.ipynb
interpreting_simulations.csv  notes_random.txt
isolate_patterns.csv          pattern_annotator.csv
method_1.png                  readymade_sim_matrix.csv
method_1.svg                  result_tarballs/
method_combinations.png       simulation_analysis.ipynb
method_image.png              simulation_contigs.csv


## <span style="color: blue;">Loading results</span>

The following block of code finds all the result files and puts them together into a single dictionary
Note we re-ran each program several times using several different databases. Most of these results aren't in the main text (aside from those pertaining to the 20191001 database) but refer to tings in the supplementary.
Also note we re-ran each database on the sub-databases of interest to make sure different sub-databases didn't interfere with one another.

### Results all contained within the result_tarballs directory and dbs all contained within the ../db_preparation directory 

Following dictionary links tarballs (for results), and database files, so that all can be analysed together
All tarballs available in result_tarballs. please run tar -xzvf * on all tarballs prior to running this code
Note we don't provide all the results as the files become large and difficult to work with in this small repo
Instead we just keep the files we need. These can all be reproduced by accding the sequence files directly and 
Then reruning each piece of software using the databases provided.


In [41]:

# Giving it where files are, note if you unpack the result tarballs somwhere else you will need to repoint the 
# "results" files

dbresult_link = {"resfinder_20191001_full":{"results":"result_tarballs/resfinderfull_20191001/",
                                          "db": "../db_preparation/resfinder_20191001/"},
                 "resfinder_20191001_blm":{"results":"result_tarballs/resfinderblm_20191001/",
                                          "db":"../db_preparation/resfinder_20191001_blm/"},
                 "resfinder_20191001_ami":{"results":"result_tarballs/resfinderami_20191001/",
                                          "db":"../db_preparation/resfinder_20191001_ami/"},
                 "resfinder_20191001_qui":{"results":"result_tarballs/resfinderqui_20191001/",
                                           "db":"../db_preparation/resfinder_20191001_qui/"},
                 "resfinder_20191001_tri":{"results":"result_tarballs/resfindertri_20191001/", 
                                          "db":"../db_preparation/resfinder_20191001_tri/"},
                 "resfinder_20191001_sul":{"results":"result_tarballs/resfindersul_20191001/", 
                                          "db":"../db_preparation/resfinder_20191001_sul/"},
                 "resfinder_20190122_full":{"results":"result_tarballs/resfinder_20190122/", 
                                          "db":"../db_preparation/resfinder_20190122/"},
                 "resfinder_20180122_full":{"results":"result_tarballs/resfinder_20180122/", 
                                          "db":"../db_preparation/resfinder_20180122/"},
                 "resfinder_20170126_full":{"results":"result_tarballs/resfinder_20170126/", 
                                          "db":"../db_preparation/resfinder_20170126/"},
                 "card_20191023_full":{"results":"result_tarballs/card_20191023/", 
                                       "db":"../db_preparation/card_20191023/"}
            }


# Loading in the output files

output_files = {}

for k in dbresult_link:
    output_files[k] = {}
    base_key = dbresult_link[k]['results'] + dbresult_link[k]['results'].split("/")[-2] + "_"
    # loading in ABRicate first
    output_files[k]["abricate"] = base_key + "abricate/"
    abricate_files = [os.path.join(root, f) for root, dirs, files 
                  in os.walk(output_files[k]["abricate"])
                 for f in files if f != "summary.tab"]
    abricate_files = {k.split("/")[-1].split("_")[0]:k for k in abricate_files}
    output_files[k]["abricate"] = abricate_files
    # Loading the KmerRes files
    output_files[k]["kmerres"] = base_key + "kmerres/"
    kmerres_files = [os.path.join(root, f) for root, dirs, files in os.walk(output_files[k]["kmerres"])
                     for f in files if ".KmerRes" in f]
    kmerres_files = {k.split("/")[-1].split("_")[0]:k for k in kmerres_files}    
    output_files[k]["kmerres"] = kmerres_files
    # The SRST2 files
    output_files[k]['srst2'] = base_key + "srst2/"
    srst2_files = [os.path.join(root, f) for root, dirs, files in os.walk(output_files[k]['srst2']) for f in files]
    srst2_files = {k.split("/")[-1].split("_")[0]:k for k in srst2_files}
    output_files[k]['srst2'] = srst2_files
    # Finally we'll put the whole ariba summary into a pandas database
    output_files[k]["ariba"] = base_key + "ariba.csv"
    ariba_names = [k.split("/")[1].split("_")[0] for k in list(pd.read_csv(output_files[k]["ariba"]).name)]
    ariba_summary = pd.read_csv(output_files[k]["ariba"]).fillna("")
    ariba_summary.index = ariba_names
    output_files[k]["ariba"] = ariba_summary
    


# Note because of the different way in which ARIBA is loaded in we also add a special reading function
def ariba_parser(s):
    s_clusters = sorted(list(set([k.split(".")[0] for k in s.index if "cluster" in k and ".match" in k])))
    s_clusters = [k for k in s_clusters if s[k+".match"] == "yes"]
    s_genes = [s[k+ ".ref_seq"] for k in s_clusters]
    return s_genes

guuids = list(output_files["resfinder_20191001_full"]["abricate"].keys())



From here on in I am going to focus on an individual database, but this is specified at the start
 
1. **resfinder_20191001_full** - The full 1st October 2019 Resfinder database
2. **resfinder_20191001_blm** - The beta-lactams only from the 1st October 2019 Resfinder database
3. **resfinder_20191001_ami** - The aminoglycosides only from the 1st October 2019 Resfinder database
4. **resfinder_20191001_qui** - The quinolones only from the 1st October 2019 Resfinder database
5. **resfinder_20191001_sul** - The sulphonomides only from the 1st October 2019 Resfinder database
6. **resfinder_20191001_tri** - The trimethoprim only from the 1st October 2019 Resfinder database
7. **resfinder_20190122_full** - The full 22nd Jan 2019 Resfinder database
8. **resfinder_20180122_full** - The full 22nd Jan 2018 Resfinder database
9. **resfinder_20170126_full** - The full 26nd Jan 2017 Resfinder database
10. **card_20191023_full** - The full 23rd October CARD database

Within these files (see above dict) there is a standard file structure
To use a different database, you just select a different database
Note for code which only applies to the resfinder_20191001_full database (The main database used in the paper) 



In [55]:

db_choice = "resfinder_20191001_full"

# Database
db = dbresult_link[db_choice]["db"]
formatted_db = db +"db_formatted.fasta"
linkfile = db + "db_link.csv"
blm_db = db + "beta-lactam.fsa"
qui_db = db + "quinolone.fsa"
ami_db = db + "aminoglycoside.fsa"
sul_db = db + "sulphonamide.fsa"
tri_db = db + "trimethoprim.fsa"

#results

abricate_files = output_files[db_choice]['abricate']
kmerres_files = output_files[db_choice]['kmerres']
srst2_files = output_files[db_choice]['srst2']
ariba_summary = output_files[db_choice]['ariba']


# 1. loading the link file
link = pd.read_csv(linkfile, index_col=0, header=None)
rlink = pd.read_csv(linkfile, index_col=1, header=None)

# 2. loading the sub files
# Note this removes duplicate Ids, this was checked as to what it did in the database formatting files
blm_db = {k.id:k for k in SeqIO.parse(blm_db, "fasta")}
qui_db = {k.id:k for k in SeqIO.parse(qui_db, "fasta")}
ami_db = {k.id:k for k in SeqIO.parse(ami_db, "fasta")}
sul_db = {k.id:k for k in SeqIO.parse(sul_db, "fasta")}
tri_db = {k.id:k for k in SeqIO.parse(tri_db, "fasta")}

# 3. loading the whole database
res_db = SeqIO.to_dict(SeqIO.parse(formatted_db, "fasta"))

# Next were going to do all vs all similarity of the resfinder database

sim_matrix = pd.DataFrame(np.zeros((len(res_db.keys()),len(res_db.keys()) )), 
                         columns = sorted(list(res_db.keys())), index=sorted(list(res_db.keys())))
# For this similarity we will use 17-mers (one of the prefixes using MASH, note several others were checked prior to this for their effects)
# Clusters marked by this are fairly similar to other k-mer sizes
res_db_kmers = {k: set([str(res_db[k].seq)[i:i+17] for i in range(len(res_db[k].seq)-16)]) for k in res_db.keys()}

def calculate_jac_sim(l1, l2):
    intersection = len(res_db_kmers[l1].intersection(res_db_kmers[l2]))
    union = len(res_db_kmers[l1].union(res_db_kmers[l2]))
    return(intersection/union)

# This code actually goes through populating this matrix, 
# However takes 30 minutes to run, and for the sake of running this code quickly on line, I will
# use a matrix I made earlier (using thie code)
if os.path.isfile("readymade"+ "_{0}_sim_matrix" .format(db_choice)) == False:
    for n in tnrange(len(res_db_kmers)):
        k  = list(res_db_kmers.keys())[n]
        for j in res_db_kmers:
            sim_matrix.loc[k][j] = calculate_jac_sim(k, j)
    sim_matrix.to_csv("readymade"+ "_{0}_sim_matrix" .format(db_choice))

sim_matrix = pd.read_csv("readymade"+ "_{0}_sim_matrix" .format(db_choice), index_col = 0)



### Setting up the analysis class


In the next section of code, the aim is to define a class which performs most of the comparisons for us. 

#### Defining useful functions
Before we set up the class we define useful functions
There are some general useful functions and also some more specific ones. 
The specific and less obvious ones are below.

**CLUSTERING**

recursive cluster => This essentially uses netrowkx to greate a graph, which links togehter elements with non-zero similarity. 
The other two functions make_tuples and name list are simplications of bits within the recursive cluster functions

**AGREEMENT**

We also use a function to define agreement
This is then useful for quick and easy ready of 
which programs have agreed together. 
Note the panel is the same one as included in a supplementary image.

![image](method_combinations.png)



#### Reading in the data
Once we have the tools to analyse the data we actually read in the data

This does the following steps
For each of ABRicate, ARIBA, KmerResistance, SRST2 we
1. read in its file
2. Pull out the TRGs it identifies and relabel them with their original names
3. Separate these into groups according to their relavent antibiotics.



#### Comparing outputs

The next stage is to define what genes we have in the sample, here we then first
1. make a combined dict of all genes found. 
2. Then separate according to clusters
3. make identification patterns for each of these clusters



This is done using an external spreadsheet (which suggests putative families for all patterns of genes seen)

In [39]:
###### USEFUL FUNCTIONS ######


###### CLUSTERING FUNCTIONS  ######


def make_tuples(l):
    # Make all possible tuples in a list
    output = []
    for i in range(len(l)):
        for j in range(len(l)):
            output.append((l[i],l[j]))
    output = sorted(list(set(output)))
    return output

def name_list(l, d):
    #rename all elements of a list
    return [d[k] for k in l]


def recursive_cluster(df, l):
    groups = {}
    # First we get all linked pais.
    for i in l:
        i_data = df.loc[i]
        i_group = [i]
        for j in l:
            if j != i: 
                if i_data[j] != 0:
                    i_group.append(j)
        i_group = sorted(i_group)
        groups[i] =  i_group
    # Assign numbers to the elements of l and then generate a dictionary to link numbers and names
    naming = {}
    reverse_naming  = {}
    m = 1
    for i in l:
        naming[i] = m
        reverse_naming[m] = i
        m += 1
    # Grouping tuples like a graph using networkx
    final_tuples = []
    for i in groups:
        final_tuples = final_tuples + make_tuples([naming[j] for j in groups[i]])
    final_tuples = sorted(list(set(final_tuples)))
    graph=nx.Graph(final_tuples)
    output = [name_list(list(c), reverse_naming) for c in nx.connected_components(graph)]
    return output

###### AGREEMENT PATTERN FUNCTIONS 

# Note for these functions they always assume the results are put in the correct order
# i.e. ABRicate, ARIBA, KmerResistance, SRST2

# First we start with a general agreement function 
def agreement_pattern(l1, l2, l3, l4):
    args = deepcopy(locals())
    arg_list = ['l1', 'l2', 'l3', 'l4']
    for key in arg_list:
        args[key] = ":".join(sorted(args[key]))
    patterns = {}
    output = []
    starting_no = 0
    for key in arg_list:
        if args[key] not in patterns:
            starting_no += 1
            patterns[args[key]] = starting_no
            output.append(starting_no)
        else:
            output.append(patterns[args[key]])
    return output
    

# Now for gene agreement, I use this program to say which genes (from a list) each method has found
def pres_bin(l1, l2):
    output = []
    for k in l1:
        if k in l2:
            output.append("1")
        else:
            output.append("0")
    return output

# Here is the agreement function again, but this time i've dropped the sort function. 
# This enables me to use the output from pres_bin directly to make the patterns as defined above 
def pres_bin_agreement_pattern(l1, l2, l3, l4):
    args = deepcopy(locals())
    arg_list = ['l1', 'l2', 'l3', 'l4']
    for key in arg_list:
        args[key] = ":".join(args[key])
    patterns = {}
    output = []
    starting_no = 0
    for key in arg_list:
        if args[key] not in patterns:
            starting_no += 1
            patterns[args[key]] = starting_no
            output.append(starting_no)
        else:
            output.append(patterns[args[key]])
    return output
    



In [54]:

# Reading in each of the data



class isolate:
    
    def __init__(self,guuid):
        self.guuid = guuid
        # First lest start with ABRicate, for this one we take everything assuming control
        # Each section of this code does similar things, 1. read the file , 2 translate the genes, 
        # 3 separate by the classes were interested in.
        self.abricate_fl = pd.read_csv(abricate_files[self.guuid], delimiter= "\t").fillna("")
        self.abricate_fl = self.abricate_fl.loc[self.abricate_fl['%COVERAGE'] > 60.0]
        self.abricate_fl = self.abricate_fl.loc[self.abricate_fl['%IDENTITY'] > 90.0]
        self.abricate_genes = sorted(list(set([link.loc[k][1] for k in list(self.abricate_fl['GENE'])])))
        self.abricate_trg = {"blm":sorted([k for k in self.abricate_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.abricate_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.abricate_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.abricate_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.abricate_genes if k in tri_db.keys()])}
        self.abricate_alltrg = sorted(list(set(self.abricate_trg['blm'] + self.abricate_trg['qui'] + 
                                               self.abricate_trg['ami'] + self.abricate_trg['sul'] +
                                              self.abricate_trg['tri'])))
        
        # Next ARIBA
        self.ariba_data = ariba_parser(ariba_summary.loc[self.guuid])
        self.ariba_genes = sorted(list(set([link.loc[k][1] for k in self.ariba_data])))
        self.ariba_trg = {"blm":sorted([k for k in self.ariba_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.ariba_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.ariba_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.ariba_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.ariba_genes if k in tri_db.keys()])}
        self.ariba_alltrg = sorted(list(set(self.ariba_trg['blm'] + self.ariba_trg['qui'] + 
                                           self.ariba_trg['ami'] + self.ariba_trg['sul'] + 
                                            self.ariba_trg['tri'])))
        
        # Next KmerResistance
        # For this one we also add in a coverage cut-off given our file doesn't seem to be able to do this reliably
        # Plus we are trying to apply the 70% cutoff as it doesn't work easily in the coverage
        # So we will re-apply this.)
        self.kmerres_fl = pd.read_csv(kmerres_files[self.guuid], delimiter = "\t").fillna("")
        self.kmerres_fl = self.kmerres_fl.loc[self.kmerres_fl.template_id > 70.0]
        self.kmerres_genes = sorted(list(set([link.loc[k][1] for k in [j for j in list(self.kmerres_fl['#Template']) if "resfindernewid" in j]])))
        self.kmerres_trg = {"blm":sorted([k for k in self.kmerres_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.kmerres_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.kmerres_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.kmerres_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.kmerres_genes if k in tri_db.keys()])}
        self.kmerres_alltrg = sorted(list(set(self.kmerres_trg['blm'] + self.kmerres_trg['qui'] + self.kmerres_trg['ami'] + self.kmerres_trg['sul'] + self.kmerres_trg['tri'] )))
        
        # Note for SRST2 we have another bit which doesn't quite work
        # It does not make a file if it finds no genes
        # Therefore we put it into a try except group

        try:
            self.srst2_fl = pd.read_csv(srst2_files[self.guuid], delimiter = "\t").fillna("")
            self.srst2_genes = sorted(list(set([link.loc[k][1] for k in list(self.srst2_fl['allele'])])))
        except:
            self.srst2_fl = "N/A"
            self.srst2_genes = []
        self.srst2_trg = {"blm":sorted([k for k in self.srst2_genes if k in blm_db.keys()]),
                             "qui":sorted([k for k in self.srst2_genes if k in qui_db.keys()]), 
                             "ami":sorted([k for k in self.srst2_genes if k in ami_db.keys()]),
                             "sul":sorted([k for k in self.srst2_genes if k in sul_db.keys()]), 
                             "tri":sorted([k for k in self.srst2_genes if k in tri_db.keys()])} 
        self.srst2_alltrg = sorted(list(set(self.srst2_trg['blm'] + self.srst2_trg['qui'] + 
                                           self.srst2_trg['ami'] + self.srst2_trg['sul'] + 
                                            self.srst2_trg['tri'])))
        
        
        ### Aggregating genes. 
        
        self.all_trg = {}
        for k in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.all_trg[k] = sorted(list(set(self.abricate_trg[k]).union(self.ariba_trg[k]).union(self.kmerres_trg[k]).union(self.srst2_trg[k])))
        self.geno_full = {"abricate":self.abricate_alltrg, "ariba":self.ariba_alltrg, 
                         "kmerres": self.kmerres_alltrg, "srst2":self.srst2_alltrg}
        
        
        
        ### Defining gene families 
        self.gene_families = {}
        for k in self.all_trg.keys():
            k_df = pd.DataFrame(np.zeros((len(self.all_trg[k]),len(self.all_trg[k]) )), 
                               columns = self.all_trg[k], index=self.all_trg[k])
            for l in k_df.index:
                for j in k_df.columns:
                    k_df.loc[l][j] = sim_matrix.loc[rlink.loc[l][0]][rlink.loc[j][0]]
            self.gene_families[k] = recursive_cluster(k_df, k_df.index)
            
            
        ### Assessing levels of agreement
        # Here we define three things, Firstly, do results agree for all genes for a particular antibiotic class
        # Then do they agree for a whole isolate
        # Then finally we do a bit more delving into the patterns of disagreement
        # Whole isolate level agreement
        
        # Firstly for eac abx class
        self.abx_patterns = {}
        for key in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.abx_patterns[key] = agreement_pattern(self.abricate_trg[key], 
                                                      self.ariba_trg[key], 
                                                      self.kmerres_trg[key], 
                                                      self.srst2_trg[key])
        self.abx_agreement = {}
        for key in ["blm", 'qui', 'ami', 'sul', 'tri']:
            self.abx_agreement[key] = (self.abx_patterns[key] == [1, 1, 1, 1])

        
        # Then for each whole isolate
        self.full_agreement = (list(set(list(self.abx_agreement.values()))) == [True])
        self.full_patterns = agreement_pattern(self.abricate_alltrg, self.ariba_alltrg, 
                                               self.kmerres_alltrg, self.srst2_alltrg)
        
        
        
        # Note there is a difference between full =  all the patterns in the 5 gene families I have
        # and full across the entiritty of resfinder , this attribute reflects this
        self.wholeresfinder_patterns = agreement_pattern(sorted(self.abricate_genes), sorted(self.ariba_genes),
                                               sorted(self.kmerres_genes), sorted(self.srst2_genes))
        self.wholeresfinder_agreement = (self.wholeresfinder_patterns == [1,1,1,1])


        
        # Lastly for each gene
        self.genes_identified = {}
        self.gene_patterns = {}
        self.gene_group = {}
        for abx in self.gene_families:
            for pat in self.gene_families[abx]:
                pat_id = ":".join(pat)
                pat_string = [pres_bin(pat, self.abricate_alltrg),
                              pres_bin(pat, self.ariba_alltrg), 
                              pres_bin(pat, self.kmerres_alltrg), 
                              pres_bin(pat, self.srst2_alltrg)]
                self.gene_patterns[pat_id] = pres_bin_agreement_pattern(pat_string[0], pat_string[1], pat_string[2], pat_string[3])
                pat_string = "|".join([":".join(i) for i in pat_string])
                self.genes_identified[pat_id] = pat_string
                self.gene_group[pat_id] = abx


# Now with the classes set up we read in everything into an isolates dict
isolates = {}

for n in tnrange(len(guuids)):
    k = guuids[n]
    x = isolate(k)
    isolates[k] = x
    print(x.gene_patterns)
                
        
        
        

{'blaCTX-M-15_1_AY044436': [1, 1, 1, 1], 'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aac(3)-IIa_1_X51534': [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_3_HQ840942': [1, 1, 1, 1]}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], "aph(3')-Ia_1_V00359": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_3_HQ840942': [1, 1, 1, 1]}
{}
{'blaTEM-1D_1_AF188200': [1, 1, 1, 1]}
{'blaTEM-1A_1_HM749966:blaTEM-1B_1_AY458016': [1, 2, 1, 1], "aadA11_2_AJ567827:aadA15_1_DQ393783:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2_2_JQ364967:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], 'sul2_2_AY034138': [1, 1, 1, 1], 'sul3_2_AJ459418': [1, 1, 1, 1], 'dfrA12_8_AM040708': [1, 1, 1, 1], 'dfrA1_10_AF203818:dfrA1_5_EU089668:dfrA1_7_

{"aph(3'')-Ib_5_AF321551": [1, 2, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_2_AY034138': [1, 1, 1, 1], 'dfrA14_4_AF393510:dfrA14_5_DQ388123': [1, 2, 2, 2]}
{"aadA11_2_AJ567827:aadA12_1_AY665771:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2b_1_D43625:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], 'sul3_2_AJ459418': [1, 1, 1, 1]}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'dfrA8_1_U10186': [1, 1, 1, 1]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA15_1_DQ393783:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2b_1_D43625:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], 'sul3_2_AJ459418': [1, 1, 1, 1]}
{'blaTEM-1D_1_AF188200': [1, 1, 1, 1], 'dfrA7_1_AB161450:dfrA7_5_AJ419170': [1, 2, 2, 2]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'qnrB19_1_EU432277:qnrB54_1_HE820727': [1, 1, 1, 2], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA15_1_DQ393783:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2_2_JQ364967:ant(3'')-Ia_1_X02340": [1, 2, 2, 3], 'sul2_

{"aadA11_2_AJ567827:aadA12_1_AY665771:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2b_1_D43625:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], "aph(3')-Ia_1_V00359": [1, 1, 1, 1], 'sul3_2_AJ459418': [1, 1, 1, 1]}
{}
{}
{'dfrA5_1_X12868:dfrA5_2_FJ001870': [1, 2, 1, 3]}
{"aadA11_2_AJ567827:aadA13_1_AY713504:aadA15_1_DQ393783:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2_2_JQ364967:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], 'sul3_2_AJ459418': [1, 1, 1, 1], 'dfrA12_8_AM040708': [1, 1, 1, 1]}
{}
{'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aadA12_1_AY665771:aadA13_1_AY713504:aadA17_1_FJ460181:aadA1_3_JQ414041:aadA24_1_AM711129:aadA24_1_DQ677333:ant(3'')-Ia_1_X02340": [1, 2, 1, 3]}
{}
{'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aadA11_2_AJ567827:aadA12_1_AY665771:aadA23_1_AJ809407:aadA2_1_NC_010870:aadA2_2_JQ364967': [1, 2, 2, 3], "aph(3')-Ia_1_V00359": [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 1, 2], 'dfrA12_8_AM040708': [1, 1, 1, 1]}
{}
{}

{"aadA11_2_AJ567827:aadA15_1_DQ393783:aadA17_1_FJ460181:aadA1_3_JQ414041:aadA1_5_JX185132:aadA24_1_AM711129:ant(3'')-Ia_1_X02340": [1, 2, 1, 3], "aph(3'')-Ib_2_AF024602:aph(3'')-Ib_3_AF321550": [1, 2, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1]}
{'blaTEM-1C_1_FJ560503': [1, 1, 1, 1]}
{'aadA4_1_Z50802': [1, 1, 1, 1], 'dfrA21_1_AY552589:dfrA22_2_KF525301:dfrA22_5_AJ968952': [1, 1, 1, 2]}
{'blaCTX-M-15_1_AY044436': [1, 1, 1, 1]}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_3_JQ414041:aadA24_1_AM711129:ant(3'')-Ia_1_X02340": [1, 2, 2, 3], 'sul3_2_AJ459418': [1, 1, 1, 1], 'dfrA1_10_AF203818:dfrA1_5_EU089668:dfrA1_7_AJ400733:dfrA1_8_X00926:dfrA1_9_AJ238350': [1, 2, 3, 4]}
{}
{'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{'blaCTX-M-15_1_AY044436': [1, 1, 1, 1], 'blaOXA-1_1_HQ170510': [1, 1,

{'blaTEM-1B_1_AY458016:blaTEM-22_1_Y17583': [1, 2, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 2, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_2_AY034138': [1, 1, 1, 1], 'dfrA14_4_AF393510:dfrA14_5_DQ388123': [1, 2, 2, 2]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_3_HQ840942': [1, 1, 1, 1], 'dfrA30_1_AM997279:dfrA5_1_X12868': [1, 1, 1, 2]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829:aph(6)-Id_4_CP000971': [1, 2, 2, 2], 'sul2_2_AY034138': [1, 1, 1, 1], 'dfrA14_1_KF921535:dfrA14_5_DQ388123': [1, 2, 2, 2]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA15_1_DQ393783:aadA1_3_JQ414041:aadA24_1_AM711129:aadA2_1_NC_010870:aadA2b_1_D43625:ant(3'')-Ia_1_X02340": [1, 2, 3, 4], 'sul2_2_AY034138': [1, 1, 1, 1], 'sul3_2_AJ459418': [1, 1, 1, 1], 'dfrA12_8_AM040708': [1, 1, 1, 1]}

...

{'blaTEM-1B_1_AY458016': [1, 1, 1, 1]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_5_JX185132:aadA24_1_AM711129:ant(3'')-Ia_1_X02340": [1, 2, 2, 3], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'sul2_3_HQ840942': [1, 1, 1, 1], 'dfrA1_5_EU089668:dfrA1_7_AJ400733:dfrA1_8_X00926:dfrA1_9_AJ238350': [1, 1, 2, 3]}
{'blaTEM-84_1_AF427130': [1, 1, 1, 1]}
{}
{'blaCTX-M-15_1_AY044436': [1, 1, 1, 1], 'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{'blaCTX-M-15_1_AY044436': [1, 1, 1, 1], 'blaOXA-1_1_HQ170510': [1, 1, 1, 1], 'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 

{'blaCTX-M-139_1_KC107824:blaCTX-M-15_1_AY044436:blaCTX-M-183_1_KX263246': [1, 2, 3, 4], 'blaOXA-1_1_HQ170510': [1, 1, 1, 1], "aac(6')-Ib-cr_1_DQ303918": [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{"aadA11_2_AJ567827:aadA12_1_AY665771:aadA17_1_FJ460181:aadA1_5_JX185132:aadA24_1_AM711129:ant(3'')-Ia_1_X02340": [1, 2, 2, 3], "aph(3'')-Ib_3_AF321550:aph(3'')-Ib_4_AF313472:aph(3'')-Ib_2_AF024602": [1, 2, 1, 3], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'dfrA1_5_EU089668:dfrA1_7_AJ400733:dfrA1_8_X00926:dfrA1_9_AJ238350': [1, 2, 3, 4]}
{}
{}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], 'aadA11_2_AJ567827:aadA12_1_AY665771:aadA23_1_AJ809407:aadA2_1_NC_010870:aadA2b_1_D43625:aadA5_1_AF137361': [1, 2, 1, 3], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], "aph(3')-Ia_1_V00359": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_3_HQ840942': [1, 1, 1, 1], 'sul3_2_AJ459418': [1, 1, 1,

{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829:aph(6)-Id_4_CP000971': [1, 2, 2, 2], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 1, 2], 'sul2_2_AY034138': [1, 1, 1, 1], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'aadA5_1_AF137361': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul1_2_U12338:sul1_5_EU780013': [1, 2, 2, 2], 'sul2_3_HQ840942': [1, 1, 1, 1], 'dfrA17_1_FJ460238': [1, 1, 1, 1]}
{'blaCTX-M-1_1_DQ915955': [1, 1, 1, 1], 'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_3_HQ840942': [1, 1, 1, 1], 'dfrA30_1_AM997279:dfrA5_1_X12868': [1, 1, 1, 2]}
{}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aadA11_2_AJ567827:aadA15_1_DQ393783:aadA17_1_FJ460181:aadA1_3_JQ414041:aadA24_1_AM711129:ant(3'')-Ia_1_X02340": [1, 2, 1, 3], 'su

{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'aac(3)-IId_1_EU022314': [1, 1, 1, 1], "aph(3'')-Ib_2_AF024602:aph(3'')-Ib_3_AF321550": [1, 2, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1]}
{'blaSHV-100_1_AM941846:blaSHV-12_1_KF976405:blaSHV-154_1_JX121121': [1, 2, 1, 3], 'blaTEM-1B_1_AY458016': [1, 1, 1, 1], 'dfrA1_7_AJ400733:dfrA1_8_X00926': [1, 1, 1, 2]}
{}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1], "aph(3'')-Ib_5_AF321551": [1, 1, 1, 1], "aph(3')-Ia_1_V00359:aph(3')-Ia_9_EU722351": [1, 2, 1, 1], 'aph(6)-Id_1_M28829': [1, 1, 1, 1], 'sul2_2_AY034138': [1, 1, 1, 1]}
{'blaTEM-1B_1_AY458016': [1, 1, 1, 1]}


KeyboardInterrupt: 

### Analysis

So with everything read in and all the identification patterns produced we next move onto producing files and images for analysis in the rest of the paper.


#### AIMS

The images we want

1. Agreement by the different methods using different databases including
 * Each of resfinder's specific databases for the 5 gene families of interest
 * The whole resfinder database
 * Possibly also adding historical databases.
 * Making the point that disagreement is highly variable even over a small window of time

2. Agreement by the different by different genes, This will include different colors of stacked bar for if only one tool disagreeing + at the top there will be a percentage value of how much of the error is artefactual.

The other data we want to analyse but not put into a picture.
Table of error suspected causes for each pattern: This would take the form of List the "pattern" found and then list which of the common sources of error cause it.
How many of the differences affect phneotype



#### Necessary outputs to get there

This explains the interim files I will want to get to this point

Figure 1: This is mostly made manually, but I will create a spreadsheet which demonstrates the numbers involved.

Figure 2: So for this I will need to 1. classify each pattern according to the parent "gene", 2. Decide if it affects resistance prediction. 3. then decide if it is artefactual using a fairly simple rule framework.

To start with we produce useful files which will be key in creating these images

**TRG pattern files**
For each TRG pattern encountered, I produce a file with how often it was seen
This will then be linked with an author provided file (which is produced manually) which derives what the "gene name" of the pattern is and whether it would change a predicted phenotype.

**Isolate Files**
For each isolate, I will produce a file which states the TRG patterns seen in the isolate and whether it was in agreement across methods. This will then be used to define whether the gene level discrepancy is "simulatable"
To be simulateable it must satisfy the following.
1. It must be discrepant
2. For at least 1 TRG identified in the discrepancy, it must be contained within the assembly completely on a single contig
3. I will finally link it with a file which describes which discrepancies are simulatable or not.
4. Note this will produced using code written on analysis rather than code which can be run locally (so as to avoid storing 2000 assemblies locally)
5. The code will be put up here but it won't be runable.

In [ ]:
#### TRG PATTERN FILES

# Aggregating the patterns from all samples
pattern_counter = {}
pattern_bymethod = {}
pattern_byabx = {}
for i in isolates:
    for k in isolates[i].genes_identified:
        pattern_key = k + "|" + isolates[i].genes_identified[k] 
        if pattern_key not in pattern_counter.keys():
            pattern_bymethod[pattern_key] = ":".join([str(j) for j in isolates[i].gene_patterns[k]])
            pattern_counter[pattern_key] = 1
            pattern_byabx[pattern_key] = isolates[i].gene_group[k]
        else:
            pattern_counter[pattern_key] += 1

print(pattern_byabx)
# writing this data into a CSV
with open("by_trg_pattern.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['trg_pattern',"abx", "number_of_isolates", "method_agreement", "overall_agreement"])
    for key in pattern_counter:
        writer.writerow([key,pattern_byabx[key], pattern_counter[key], pattern_bymethod[key], pattern_bymethod[key]=="1:1:1:1"])



In [ ]:
#### ISOLATE files:

with open("isolate_patterns.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    writer.writerow(["isolate", "pattern", "agreement"])
    for k in isolates:
        for pat in isolates[k].genes_identified:
            writer.writerow([k, pat, ("0" not in isolates[k].genes_identified[pat])])
        

In [ ]:
### For presentation stuff


annotated_patterns = pd.read_csv("pattern_annotator.csv")
gene_output = pd.read_csv("by_trg_pattern.csv")
pat_data = gene_output.merge(annotated_patterns, on="trg_pattern")
simulation_data = pd.read_csv("interpreting_simulations.csv")
print(simulation_data.head())

def met_agr(l):
    if l == "1:1:1:1":
        return "0"
    if l == "1:2:2:2":
        return "1"
    if l == "1:2:1:1":
        return "2"
    if l == "1:1:2:1":
        return "3"    
    if l == "1:1:1:2":
        return "4"
    else:
        return "5"

pat_dict = {}

pg_dict = {}
artefact_dict = {}

for k in range(len(pat_data)):
    k_data = pat_data.iloc[k]
    k_pat = k_data.trg_pattern.split("|")[0]
    sim_dat = (True in list(simulation_data.loc[simulation_data.pattern == k_pat].overall))
    if k_data.method_agreement == "1:1:1:1":
        k_status = "0"
    elif sim_dat == True:
        k_status = "1"
    else:
        k_status = "2"
    if k_data.gene_name not in pat_dict.keys():
        pat_dict[k_data.gene_name] = {str(i):0 for i in range(6)}
        artefact_dict[k_data.gene_name]  = {str(i):0 for i in range(3)}
    pat_dict[k_data.gene_name][met_agr(k_data.method_agreement)] += k_data.number_of_isolates
    artefact_dict[k_data.gene_name][k_status] += k_data.number_of_isolates
    pg_dict[k_data.gene_name] = k_data.abx_x


def ad_sum(d):
    try:
        return d["1"]/(d["1"]+d["2"])
    except ZeroDivisionError:
        return -1

for k in pg_dict:
    pg_dict[k] = (pg_dict[k], sum(pat_dict[k].values()),round(ad_sum(artefact_dict[k]), 2))

print(pat_dict)
for k in sorted(pg_dict.keys(), key = lambda a: (pg_dict[a][0],pg_dict[a][1]) , reverse = True):
    print(k, pg_dict[k])

In [ ]:
f  = plt.figure(figsize=(10, 5), dpi=300)
ax1 = plt.subplot2grid((1,1),(0,0), rowspan = 1 , colspan=1)

f_keys = ["blaTEM", "blaCTX-M-1", "blaOXA-1","blaCMY", "blaSHV","blaCTX-M-9" , 
         "aph(6)-Id","aph(3'')-Ib", "ant(3'')-Ia", "aadA5", "aac(3)-IIa", "aac(6')-Ib", "aph(3')-Ia", 
         "aac(3)-IV", "aph(4)-Ia", 
         "qnrS",
         "dfrA7","dfrA1", "drfA12", "dfrA14" , "dfrA5", 
         "sul2","sul1", "sul3" ]
xs = range(len(f_keys))
f_0vals = [pat_dict[k]['0'] for k in f_keys]
f_1vals = [pat_dict[k]['1'] for k in f_keys]
f_2vals = [pat_dict[k]['2'] for k in f_keys]
f_3vals = [pat_dict[k]['3'] for k in f_keys]
f_4vals = [pat_dict[k]['4'] for k in f_keys]
f_5vals = [pat_dict[k]['5'] for k in f_keys]
def convert_numbers(l):
    out_list = []
    for k in l:
        if k != -1:
            out_list.append(str(int(100*k)) + "%")
        else:
            out_list.append("N/A")
    return out_list

f_numbers = convert_numbers([pg_dict[k][2] for k in f_keys])
print(f_numbers)
width = 0.5

ax1.bar(xs, f_0vals, width)
ax1.bar(xs, f_1vals, width,
             bottom=f_0vals, label="ABRicate discrepant")
ax1.bar(xs, f_2vals, width,
             bottom= [f_0vals[i]+ f_1vals[i] for i in range(len(f_0vals))], label="ARIBA discrepant")
ax1.bar(xs, f_3vals, width,
             bottom= [f_0vals[i]+ f_1vals[i]+f_2vals[i] for i in range(len(f_0vals))], label="KmerResistance discrepant")
ax1.bar(xs, f_4vals, width,
             bottom= [f_0vals[i]+ f_1vals[i]+f_2vals[i]+f_3vals[i] for i in range(len(f_0vals))], label="SRST2 discrepant")
ax1.bar(xs, f_5vals, width,
             bottom= [f_0vals[i]+ f_1vals[i]+f_2vals[i]+f_3vals[i]+f_4vals[i] for i in range(len(f_0vals))], label="Multiple discrepant")

ax1.set_xticks(range(len(f_keys)))
ax1.set_xticklabels(f_keys, rotation=90)
ax1.set_yticklabels([], rotation=90)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
